In [129]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
%reload_ext autoreload

In [131]:
import pandas as pd
import seaborn as sns
import spacy

In [132]:
speeches = pd.read_csv('millercenter_speeches_df.csv', index_col=0).reset_index(drop=True)

In [133]:
import sys
sys.path.append('/Users/susan_jose/Jose/springboard/nlp_political_speeches/Text_Pre-processing/')
from text_preprocessing_helpers import expand_contractions, remove_special_characters

In [134]:
def remove_characters(text):
    import re
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[\[\]]', ' ', text)
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, ' ', text)
    return text

In [135]:
# Expand contractions using DJ's function
speeches['Transcript'] = speeches['Transcript'].apply(lambda x : expand_contractions(x))

In [136]:
speeches['Transcript'] = speeches['Transcript'].apply(lambda x : remove_characters(x))

In [66]:
# No need to do POS, dependency parsing, NER or other pre-processing

In [6]:
# No need to convert accented characters for this type of text

In [65]:
# Using SpaCy to lemmatize transcripts
# Remove stop-words

In [137]:
nlp = spacy.load('en')

In [138]:
def lemmatize_speech(speech):
    text = nlp(speech)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [139]:
def remove_stopwords(speech):
    data = nlp(speech)
    data = ' '.join([word.text for word in data if not word.is_stop ])
    return data

In [140]:
speeches['Transcript'] = speeches['Transcript'].apply(lambda x : lemmatize_speech(x))

In [141]:
# Convert to lowercase
speeches['Transcript'] = speeches['Transcript'].str.lower()

In [142]:
speeches['Transcript'] = speeches['Transcript'].apply(lambda x : remove_stopwords(x))

In [143]:
# Replace more than one space with just one
speeches['Transcript'] = speeches['Transcript'].str.replace('[ ]{2,}', ' ')

In [144]:
def remove_non_ascii(speech):
    data = nlp(speech)
    data = ' '.join([word.text for word in data if word.is_ascii ])
    return data

In [145]:
speeches['Transcript'] = speeches['Transcript'].apply(lambda x : remove_non_ascii(x))

In [146]:
with open('democratic_politicians.txt', 'r') as f:
    dem = f.read().split('\n')
with open('republican_politicians.txt', 'r') as g:
    rep = g.read().split('\n')

In [147]:
# Create list with affiliation per row
aff = ['Republican' if p in rep else 'Democrat' if p in dem else 'Missing' for p in speeches['President']]
#Assign affiliation to rows
speeches['Affiliation'] = aff

In [148]:
speeches.loc[speeches['Affiliation'] == 'Democrat', 'Label'] = int(1)
speeches.loc[speeches['Affiliation'] == 'Republican', 'Label'] = int(0)
speeches['Label'] = speeches['Label'].astype(int)

In [152]:
speeches.to_csv('../speeches_millercenter_preprocessed.csv', index=False)

In [151]:
speeches['Transcript'][0]

'34th time speak oval office year soon time want share thought save long time honor life president write past week thank nancy grateful opportunity serve thing presidency somewhat apart spend lot time fast car drive people tint glass parent hold child wave late return time want stop reach glass connect maybe little tonight people ask feel leave fact sweet sorrow sweet california ranch freedom sorrow goodbye course leave beautiful place know hall stair office white house president family live favorite window like stand look early morning view ground washington monument mall jefferson memorial morning humidity low past jefferson river potomac virginia shore view lincoln smoke rise battle bull run prosaic thing grass bank morning traffic people way work sailboat river think bit window reflect past year mean mean image come mind like refrain nautical small story big ship refugee sailor early 80 height boat people sailor hard work carrier midway patrol south china sea sailor like american s